<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_5/blob/main/Session5_Assignment_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import pandas as pd
import numpy as np
import csv

!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 10.3MB/s 
     |████████████████████████████████| 1.0MB 41.7MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 61kB 11.0MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=b9439bd61741ab72b94168adfb9bd95d242c40899ee2648b86c3e01febc6f7f7
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [2]:
import random
import googletrans
from googletrans import Translator

def translate(sentence_translated):
  # translator = Translator(service_urls=['translate.googleapis.com'])
  translator = Translator()
  # sentence = ['']

  # available_langs = list(googletrans.LANGUAGES.keys()) 
  # trans_lang = random.choice(available_langs) 
  # print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  sentence_translated = pd.DataFrame(sentence_translated.iloc[0:400,:]).reset_index().drop(columns=['index'])

  trans_array = np.array(sentence_translated['sentence'])
  # int(len(trans_array)/100)

  for i in range(400):
    translations = translator.translate(trans_array[i],src='en', dest='czech') 
    #print(translations)
    t_text = translations.text
    print('czech: ',t_text)

    translations_en_random = translator.translate(t_text, src='czech', dest='en') 
    en_text = translations_en_random.text
    print('Eng: ',en_text)
    trans_array[i] = en_text


  sentence_translated['sentence'] = trans_array
  return sentence_translated



In [3]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [4]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [6]:
sentences = pd.read_csv('/content/datasetSentences.txt',sep='\t')
sentences.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [7]:
dictonary = pd.read_csv('/content/dictionary.txt',sep='|', names=('Phrase','Pharase_Id'))
dictonary.head()

,Phrase,Pharase_Id
0,!,0
1,! ',22935
2,! '',18235
3,! Alas,179257
4,! Brilliant,22936


In [8]:
dataSplit = pd.read_csv('/content/datasetSplit.txt', sep=',')
dataSplit.head()

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


In [9]:
sentiment = pd.read_csv('/content/sentiment_labels.txt',sep='|')
sentiment.head()

,phrase ids,sentiment values
0,0,0.50000
1,1,0.50000
2,2,0.44444
3,3,0.50000
4,4,0.42708


In [10]:
sentiment_class = []
for i in sentiment['sentiment values'] :
  if i >=0 and i<0.2:
    sentiment_class.append(1)
  elif i>=0.2 and i<0.4:
    sentiment_class.append(2)
  elif i>=0.4 and i<0.6:
    sentiment_class.append(3)
  elif i>=0.6 and i<0.8:
    sentiment_class.append(4)
  else:
    sentiment_class.append(5)

sentiment['sentiment values'] = sentiment_class


In [11]:
sentiment.describe()

,phrase ids,sentiment values
count,239232.000000,239232.000000
mean,119615.500000,3.062391
std,69060.474137,0.911298
min,0.000000,1.000000
25%,59807.750000,3.000000
50%,119615.500000,3.000000
75%,179423.250000,4.000000
max,239231.000000,5.000000


In [12]:
## This idea of sqlite and join is taken from github ref:
## https://gist.github.com/soaxelbrooke/226480b00a86dad2e6f8bbb2b28c07a2
import sqlite3
quantize = '--quantize' in sys.argv
conn = sqlite3.connect(':memory:')
conn.execute('CREATE TABLE sentences (id LONG PRIMARY KEY, sentence TEXT)')
conn.execute('CREATE TABLE dictionary (phrase TEXT,id LONG PRIMARY KEY)')
conn.execute('CREATE TABLE sentiments (phrase_id LONG PRIMARY KEY, sentiment {})'.format('INT' if quantize else 'FLOAT'))
conn.execute('CREATE TABLE splits (sentence_id LONG PRIMARY KEY, partition INT)')

In [13]:
conn.execute('CREATE INDEX sentences_sentence_idx ON sentences (sentence);')
conn.execute('CREATE INDEX dictionary_phrase_idx ON dictionary (phrase);')

In [14]:
sentences.items()

<generator object DataFrame.items at 0x7f14e43b64d0>

In [15]:
# sentences.values.tolist()

In [16]:
conn.executemany('INSERT INTO sentences VALUES (?, ?)', sentences.values.tolist())
conn.executemany('INSERT INTO dictionary VALUES (?, ?)', dictonary.values.tolist())
conn.executemany('INSERT INTO sentiments VALUES (?, ?)', sentiment.values.tolist())
conn.executemany('INSERT INTO splits VALUES (?, ?)', dataSplit.values.tolist())

In [17]:
conn.commit()
crs = conn.cursor()

crs.execute('''
        SELECT
            partition,
            sentence,
            sentiment
        FROM sentences 
            JOIN dictionary ON sentence = phrase
            JOIN sentiments ON dictionary.id = phrase_id
            JOIN splits ON sentences.id = sentence_id
        ''')

In [18]:
dataset = crs.fetchall()

In [19]:
for fname, partition in [['train', 1], ['test', 2], ['dev', 3]]:
    with open(fname + '.csv', 'w') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['sentence', 'sentiment'])
        for row in dataset:
            if row[0] == partition:
                writer.writerow(row[1:])

In [35]:
train_data = pd.read_csv('/content/train.csv')
train_data.head()


,sentence,sentiment
0,... a bland murder-on-campus yawner .,1.0
1,... the picture 's cleverness is ironically mu...,3.0
2,classic cinema served up with heart and humor,5.0
3,"entertaining enough , but nothing new",2.0
4,"insightfully written , delicately performed",5.0


In [21]:
train_data.shape

(8117, 2)

In [47]:
# train_data['sentiment'] = train_data['sentiment'].astype(str)

In [22]:
train_data.sentiment.value_counts()

4.0    2216
2.0    2092
3.0    1549
5.0    1230
1.0    1030
Name: sentiment, dtype: int64

In [23]:
train_data.head()

,sentence,sentiment
0,... a bland murder-on-campus yawner .,1.0
1,... the picture 's cleverness is ironically mu...,3.0
2,classic cinema served up with heart and humor,5.0
3,"entertaining enough , but nothing new",2.0
4,"insightfully written , delicately performed",5.0


In [56]:

def delete(sent_delete):
  del_array = np.array(sent_delete['sentence'])
  for i in range(len(del_array)):
    words = str(del_array[i]).split()
    word_selected = random.choice(words)
    if word_selected in ['`','~','!','@','#','$','%','^','&','*','(',')','-','_','+','=','{','}','[',']',':',';','|','\\',',','.','<','>','/','\'','?']:
      continue
    word_deleted = random_deletion(word_selected, 0.4)
    del_word = ''
    for k in word_deleted:
      del_word+=k
    del_array[i] = str(del_array[i]).replace(word_selected,del_word)
  sent_delete['sentence'] = del_array
  return sent_delete

sentence_delete = pd.DataFrame(train_data.iloc[:,:]).reset_index().drop(columns=['index'])
print(sentence_delete.head())
sentence_delete = delete(sentence_delete)

                                            sentence  sentiment
0              ... a bland murder-on-campus yawner .        1.0
1  ... the picture 's cleverness is ironically mu...        3.0
2      classic cinema served up with heart and humor        5.0
3              entertaining enough , but nothing new        2.0
4        insightfully written , delicately performed        5.0


In [57]:
print(sentence_delete.head())

                                            sentence  sentiment
0              ... a bland murder-on-campus yawner .        1.0
1  ... the picture 's evrns is ironically muted b...        3.0
2        classic cinema served up wh heart and humor        5.0
3                 entertaining eno , but nothing new        2.0
4           ingtfully written , delicately performed        5.0


In [58]:
print(sentence_delete.head(10))

                                            sentence  sentiment
0              ... a bland murder-on-campus yawner .        1.0
1  ... the picture 's evrns is ironically muted b...        3.0
2        classic cinema served up wh heart and humor        5.0
3                 entertaining eno , but nothing new        2.0
4           ingtfully written , delicately performed        5.0
5                Yet the act s still charming here .        4.0
6  You 'd think by now America would have had eno...        3.0
7               ... a essn in prehistoric hilarity .        3.0
8  ... a trashy little bit of fluff stuffed with ...        3.0
9  ... a triumph of emotionally and narratively c...        5.0


In [27]:
sentence_swap = pd.DataFrame(train_data.iloc[:,:]).reset_index().drop(columns=['index']) 
print(sentence_swap.head())
sentence_swap['sentence'] = random_swap(sentence_swap['sentence'])
sentence_swap.head(6)

                                            sentence  sentiment
0              ... a bland murder-on-campus yawner .        1.0
1  ... the picture 's cleverness is ironically mu...        3.0
2      classic cinema served up with heart and humor        5.0
3              entertaining enough , but nothing new        2.0
4        insightfully written , delicately performed        5.0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,sentence,sentiment
0,... a bland murder-on-campus yawner .,1.0
1,... the picture 's cleverness is ironically mu...,3.0
2,classic cinema served up with heart and humor,5.0
3,"entertaining enough , but nothing new",2.0
4,"insightfully written , delicately performed",5.0
5,Yet the act is still charming here .,4.0


In [28]:
print(train_data.shape)
print(sentence_delete.shape)
print(sentence_swap.shape)
sentence_translated = train_data
sentence_translated = translate(sentence_translated)
print(sentence_translated.shape)

(8117, 2)
(8117, 2)
(8117, 2)
czech:  ... nevýrazný zející vrah na akademické půdě.
Eng:  ... a faint yawning killer on campus.
czech:  ... chytrost obrazu je ironicky ztlumena samotnými lidmi, kteří ho chtějí rozzářit.
Eng:  ... the cleverness of the image is ironically dimmed by the people themselves who want to brighten it.
czech:  klasické kino servírované srdcem a humorem
Eng:  classic cinema served with heart and humor
czech:  dostatečně zábavné, ale nic nového
Eng:  fun enough, but nothing new
czech:  nápadně napsané, jemně provedené
Eng:  conspicuously written, finely executed
czech:  Tento akt je zde stále okouzlující.
Eng:  This act is still charming here.
czech:  Mysleli byste si, že Amerika by už měla dost odvážných britských výstředníků se zlatými srdíčky.
Eng:  You would think that America would already have enough brave British eccentricities with golden hearts.
czech:  ... lekce prehistorické veselosti.
Eng:  ... lessons of prehistoric merriment.
czech:  ... kýčovitý ko

In [29]:
train_data_1 = pd.concat([train_data,sentence_swap,sentence_delete,sentence_translated]).reset_index().drop(columns=['index'])
train_data_1.head()

,sentence,sentiment
0,... a bland murder-on-campus yawner .,1.0
1,... the picture 's cleverness is ironically mu...,3.0
2,classic cinema served up with heart and humor,5.0
3,"entertaining enough , but nothing new",2.0
4,"insightfully written , delicately performed",5.0


In [30]:
train_data_1.shape

(24751, 2)

In [59]:
#train_data = train_data_1
train_data_1.sentiment.value_counts()

4.0    6855
2.0    6292
3.0    4695
5.0    3817
1.0    3092
Name: sentiment, dtype: int64

In [60]:
new_sentence_delete_1 = train_data[train_data.sentiment==1.0].reset_index().drop(columns=['index'])
new_sentence_delete_2 = train_data[train_data.sentiment==5.0].reset_index().drop(columns=['index'])
new_sentence_delete_3 = train_data[train_data.sentiment==3.0].reset_index().drop(columns=['index'])
new_sentence_delete = pd.concat([new_sentence_delete_1,new_sentence_delete_2,new_sentence_delete_3]).reset_index().drop(columns=['index'])
new_sentence_delete.head()

,sentence,sentiment
0,... a bland murder-on-campus yawner .,1.0
1,A decided lack of spontaneity in its execution...,1.0
2,... the film 's considered approach to its sub...,1.0
3,A devastating indictment of unbridled greed an...,1.0
4,"All comedy is subversive , but this unrelentin...",1.0


In [61]:
new_sentence_delete = delete(new_sentence_delete)
new_sentence_delete.head()

,sentence,sentiment
0,... a bad murder-on-campus yawner .,1.0
1,A decided lack of otaiy in its execution and a...,1.0
2,... the film 's considered approach to its sub...,1.0
3,A devastating indictment of nbid greed and mat...,1.0
4,"All comedy is subversive , but this unrelentin...",1.0


In [65]:
train_data = pd.concat([train_data_1,new_sentence_delete]).reset_index().drop(columns=['index'])
print(train_data.head())
print(train_data.shape)
print(train_data.sentiment.value_counts())

                                            sentence  sentiment
0              ... a bland murder-on-campus yawner .        1.0
1  ... the picture 's cleverness is ironically mu...        3.0
2      classic cinema served up with heart and humor        5.0
3              entertaining enough , but nothing new        2.0
4        insightfully written , delicately performed        5.0
(28560, 2)
4.0    6855
2.0    6292
3.0    6244
5.0    5047
1.0    4122
Name: sentiment, dtype: int64


In [66]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [67]:
sentence = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
sentiments = torchtext.legacy.data.Field(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [68]:
fields = [('sentence', sentence),('sentiments',sentiments)]

In [69]:
example = [torchtext.legacy.data.Example.fromlist([train_data.sentence[i],train_data.sentiment[i]], fields) for i in range(train_data.shape[0])] 

In [70]:
trainDataset = torchtext.legacy.data.Dataset(example, fields)

In [71]:
valid_data = pd.read_csv('/content/dev.csv')
valid_data.head()

,sentence,sentiment
0,... a hollow joke told by a cinematic gymnast ...,2.0
1,"... Mafia , rap stars and hood rats butt their...",4.0
2,... a story we have n't seen on the big screen...,4.0
3,"... an otherwise intense , twist-and-turn thri...",4.0
4,... takes the beauty of baseball and melds it ...,4.0


In [72]:
valid_data.shape

(1044, 2)

In [73]:
# valid_data['sentiment'] = valid_data['sentiment'].astype(str)


In [74]:
valid_data.sentiment.value_counts()

2.0    276
4.0    259
3.0    219
5.0    158
1.0    132
Name: sentiment, dtype: int64

In [75]:
example = [torchtext.legacy.data.Example.fromlist([valid_data.sentence[i],valid_data.sentiment[i]], fields) for i in range(valid_data.shape[0])] 

In [76]:
validDataset = torchtext.legacy.data.Dataset(example, fields)

In [77]:
vars(trainDataset.examples[10])

{'sentence': ['...',
  'begins',
  'on',
  'a',
  'high',
  'note',
  'and',
  'sustains',
  'it',
  'beautifully',
  '.'],
 'sentiments': 4.0}

In [78]:
sentence.build_vocab(trainDataset)
sentiments.build_vocab(trainDataset)

In [79]:
print('Size of input vocab : ', len(sentence.vocab))
print('Size of label vocab : ', len(sentiments.vocab))
print('Top 10 words appreared repeatedly :', list(sentence.vocab.freqs.most_common(10)))
print('Labels : ', sentiments.vocab.stoi)

Size of input vocab :  20642
Size of label vocab :  6
Top 10 words appreared repeatedly : [('.', 26872), (',', 23632), ('the', 19655), ('a', 14662), ('of', 14497), ('and', 14409), ('to', 9879), ('-', 8952), ('is', 8373), ("'s", 8157)]
Labels :  defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f148868f650>>, {'<unk>': 0, 4.0: 1, 2.0: 2, 3.0: 3, 5.0: 4, 1.0: 5})


In [80]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [89]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((trainDataset, validDataset), batch_size = 128, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

In [90]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(sentence.vocab.stoi, tokens)

In [97]:
# Define hyperparameters
size_of_vocab = len(sentence.vocab)
embedding_dim = 200
num_hidden_nodes = 300
num_output_nodes = 6
num_layers = 5
dropout = 0.7

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [98]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(20642, 200)
  (encoder): LSTM(200, 300, num_layers=5, batch_first=True, dropout=0.7)
  (fc): Linear(in_features=300, out_features=6, bias=True)
)
The model has 7,622,206 trainable parameters


In [99]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [100]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence  
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        # compute the loss
        loss = criterion(predictions, batch.sentiments)        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.sentiments)   
        # backpropage the loss and compute the gradients
        loss.backward()       
        # update the weights
        optimizer.step()      
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [101]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.sentiments)
            acc = binary_accuracy(predictions, batch.sentiments)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [102]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.744 | Train Acc: 25.83%
	 Val. Loss: 1.716 |  Val. Acc: 30.76% 

	Train Loss: 1.686 | Train Acc: 34.55%
	 Val. Loss: 1.691 |  Val. Acc: 32.53% 

	Train Loss: 1.618 | Train Acc: 42.59%
	 Val. Loss: 1.687 |  Val. Acc: 32.41% 

	Train Loss: 1.548 | Train Acc: 50.40%
	 Val. Loss: 1.710 |  Val. Acc: 29.48% 

	Train Loss: 1.487 | Train Acc: 56.61%
	 Val. Loss: 1.706 |  Val. Acc: 31.42% 

	Train Loss: 1.440 | Train Acc: 61.38%
	 Val. Loss: 1.704 |  Val. Acc: 31.56% 

	Train Loss: 1.397 | Train Acc: 65.41%
	 Val. Loss: 1.716 |  Val. Acc: 30.92% 

	Train Loss: 1.372 | Train Acc: 67.71%
	 Val. Loss: 1.707 |  Val. Acc: 31.42% 

	Train Loss: 1.351 | Train Acc: 69.83%
	 Val. Loss: 1.709 |  Val. Acc: 32.03% 

	Train Loss: 1.320 | Train Acc: 73.53%
	 Val. Loss: 1.714 |  Val. Acc: 31.82% 



In [103]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_sentence(sentence):
    
    categories = {0: "very negative", 1:"negative", 2:"neutral", 3:"positive", 4:"very positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [104]:
classify_sentence("The notion that bombing buildings is the funniest thing in the world goes entirely unexamined in this startlingly unfunny comedy .")

'negative'

In [105]:
classify_sentence("Visually rather stunning , but ultimately a handsome-looking bore , the true creativity would have been to hide Treasure Planet entirely and completely reimagine it .")

'very positive'

In [106]:
classify_sentence("Miller is playing so free with emotions , and the fact that children are hostages to fortune , that he makes the audience hostage to his swaggering affectation of seriousness .")

'neutral'

In [107]:
classify_sentence("Schaeffer has to find some hook on which to hang his persistently useless movies , and it might as well be the resuscitation of the middle-aged character .")

'neutral'

In [108]:
classify_sentence("There 's just no currency in deriding James Bond for being a clichÃ©d , doddering , misogynistic boy 's club .")

'positive'

In [109]:
classify_sentence("When the film ended , I felt tired and drained and wanted to lie on my own deathbed for a while .")

'very positive'

In [110]:
classify_sentence("Full of witless jokes , dealing in broad stereotypes and outrageously unbelievable scenarios , and saddled with a general air of misogyny")

'neutral'

In [111]:
classify_sentence("The film 's hackneyed message is not helped by the thin characterizations , nonexistent plot and pretentious visual style .")

'neutral'

In [112]:
classify_sentence("Detox is ultimately a pointless endeavor .")

'neutral'

In [113]:
classify_sentence("Too much of the humor falls flat .")

'neutral'

In [114]:
classify_sentence("No way I can believe this load of junk .")

'negative'